# XBRL Values to Dataframe
Get Single Values from XBRL Statement

In [ ]:
pip install arelle isodate requests_random_user_agent feedparser

In [4]:
from arelle import Cntlr
from arelle import ModelManager
import requests_random_user_agent
import csv
import pandas as pd
import feedparser
import sys
import requests
from datetime import datetime
import lxml.html as LH 
import zipfile
import time
import io
import os

#Change CIK for looking into another Company
cikkey = '0000789019'
year = 2015


#Check if Year is in the Range of Supported Years and check if File is in xml or html format

if year < 2009:
  print('Year is not supported by this Script please enter a Year above 2008')
  sys.exit()

#Get all Companyfilings reported to the SEC 
Feed = feedparser.parse(requests.get("https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK={}&type=10-k%25&dateb=&owner=exclude&start=0&count=40&output=atom".format(cikkey)).text)

#Get the Year the User specified before
for index in range(len(Feed.entries)):
    typeofreport = Feed.entries[index]['filing-type']
    reportedyear = datetime.strptime(Feed.entries[index]['filing-date'], '%Y-%d-%M').year - 1 
    if reportedyear == year and typeofreport == '10-K':
      IndexofYear = index
      break
    else:
      IndexofYear = 0
time.sleep(1)
#Build URL to access DIrectory AND Store Accession Number
acn = Feed.entries[IndexofYear]['accession-number'].replace("-","")
createdlink = 'https://www.sec.gov/Archives/edgar/data/{}/{}/'.format(cikkey, acn)
print(createdlink)
htmlroot = requests.get(createdlink).text
htmlroot = LH.fromstring(htmlroot)
link = htmlroot.xpath('.//a[contains(text(),".zip")]/@href')[0]
time.sleep(1)
#Download XBRL Files
if not os.path.exists(acn):
    os.makedirs(acn)
zipfiledl = requests.get('https://www.sec.gov/' + link)
zip = zipfile.ZipFile(io.BytesIO(zipfiledl.content))
zip.extractall(acn)
XBRLFILE = os.listdir(path=acn)

for element in XBRLFILE:
  if  '_cal' not in element and '_def' not in element and '_lab' not in element and '_pre' not in element and '.xml' in element or '.htm' in element and '10k_'in element or '.htm' in element and '10q_'in element:
    XBRLFILE = element
print(XBRLFILE)

d = {
    'ID': [],
    'Prefix': [],
    'Name': [],
    'Context': [],
    'Year': [],
    'FromTable': [],
    'SourceLine': [],
    'Unit': [],
    'Decimals': [],
    'EffectiveValue': [],
    'Value': [],
}

xbrl = Cntlr.Cntlr().modelManager.load(acn + '/' + XBRLFILE)
for fact in xbrl.facts:
  d['ID'].append(fact.id)
  d['Name'].append(fact.qname)
  d['Context'].append(fact.contextID)
  d['FromTable'].append(fact.attrib)
  d['SourceLine'].append(fact.sourceline)
  d['Prefix'].append(fact.prefix)
  d['EffectiveValue'].append(fact.effectiveValue)
  d['Unit'].append(fact.unitID)
  d['Decimals'].append(fact.decimals)
  d['Value'].append(fact.value)
  if year >= 2016:
    sub = fact.attrib['contextRef'].split("_")[len(fact.attrib['contextRef'].split("_")) - 1]
    sub = sub[6:8] + '-' + sub[4:6] + '-' + sub[0:4]
    d['Year'].append(sub)
  if year < 2016:
    sub = fact.attrib['contextRef'].split("_")[4]
    sub = sub[6:8] + '-' + sub[4:6] + '-' + sub[0:4]
    d['Year'].append(sub)
df = pd.DataFrame(data=d)
df = df.drop_duplicates(subset=['Context'])
df


https://www.sec.gov/Archives/edgar/data/0000789019/000119312516662209/
msft-20160630.xml


,ID,Prefix,Name,Context,Year,FromTable,SourceLine,Unit,Decimals,EffectiveValue,Value
0,id_8234187_02F29457-607B-4A9C-89AD-BF98626D5E0...,us-gaap,us-gaap:BusinessCombinationRecognizedIdentifia...,eol_PE8528----1610-K0009_STD_0_20141106_0_1333...,06-11-2014,"[contextRef, unitRef, decimals, id]",9,iso4217_USD,-6,"928,000,000",928000000
2,id_8234187_A53ACEAD-2E70-4B21-A91C-5E0D1DD7871...,us-gaap,us-gaap:StockRepurchaseProgramAuthorizedAmount1,eol_PE8528----1610-K0009_STD_0_20130916_0,16-09-2013,"[contextRef, unitRef, decimals, id]",11,iso4217_USD,INF,"40,000,000,000",40000000000
3,id_8234187_F41396E5-A465-40F4-94C2-856A5DC840E...,us-gaap,us-gaap:DebtInstrumentFaceAmount,eol_PE8528----1610-K0009_STD_0_20151130_0_1324...,30-11-2015,"[contextRef, unitRef, decimals, id]",12,iso4217_USD,-8,"13,000,000,000",13000000000
4,id_8234187_DD1F40E2-2713-4076-BAD8-D0CEBF60B1D...,us-gaap,us-gaap:BusinessCombinationRecognizedIdentifia...,eol_PE8528----1610-K0009_STD_0_20140425_0_1331...,25-04-2014,"[contextRef, unitRef, decimals, id]",13,iso4217_USD,-6,"1,500,000,000",1500000000
5,id_8234187_DD1F40E2-2713-4076-BAD8-D0CEBF60B1D...,us-gaap,us-gaap:BusinessCombinationRecognizedIdentifia...,eol_PE8528----1610-K0009_STD_0_20140425_0_1331...,25-04-2014,"[contextRef, unitRef, decimals, id]",14,iso4217_USD,-6,"2,493,000,000",2493000000
...,...,...,...,...,...,...,...,...,...,...,...
2093,id_8234187_3D9E7BF8-CC98-4F0A-8F72-DB122D72ADF...,us-gaap,us-gaap:IncomeTaxExaminationYearUnderExamination,eol_PE8528----1610-K0009_STD_92_20150930_0_133...,30-09-2015,"[contextRef, id]",35819,None,None,2009,2009
2094,id_8234187_3D9E7BF8-CC98-4F0A-8F72-DB122D72ADF...,us-gaap,us-gaap:IncomeTaxExaminationYearUnderExamination,eol_PE8528----1610-K0009_STD_92_20150930_0_133...,30-09-2015,"[contextRef, id]",35820,None,None,2007,2007
2095,id_8234187_1EE0269D-B164-44AB-8831-2BB26FB624B...,us-gaap,us-gaap:SalesRevenueNet,eol_PE8528----1610-K0009_STD_92_20150930_0_133...,30-09-2015,"[contextRef, unitRef, decimals, id]",35821,iso4217_USD,-8,"1,300,000,000",1300000000
2096,id_8234187_4605D0CE-DB19-4F65-A7E4-33661119935...,us-gaap,us-gaap:DividendsPayableDateDeclaredDayMonthAn...,eol_PE8528----1610-K0009_STD_92_20151231_0,31-12-2015,"[contextRef, id]",35822,None,None,2015-12-02,2015-12-02


# XBRL to JSON
Get every Table from Financial Statment formated as JSON

In [6]:
!git clone https://github.com/Arelle/Arelle

Cloning into 'Arelle'...
remote: Enumerating objects: 29507, done.
remote: Counting objects: 100% (436/436), done.
remote: Compressing objects: 100% (227/227), done.
remote: Total 29507 (delta 268), reused 329 (delta 207), pack-reused 29071
Receiving objects: 100% (29507/29507), 114.08 MiB | 37.98 MiB/s, done.
Resolving deltas: 100% (21369/21369), done.


In [36]:
import requests_random_user_agent
import csv
import pandas as pd
import feedparser
import sys
import requests
from datetime import datetime
import lxml.html as LH 
import zipfile
import time
import io
import os
from datetime import date
import xml.etree.ElementTree as ET
from lxml import etree
from datetime import datetime, timedelta, date
from dateutil.relativedelta import relativedelta
from dateutil.parser import parse
import json
import shutil

#Change CIK for looking into another Company
cikkey_list = ['000320193']
year = 2015
everyyear = True
formtype = "10-K"
###
noreport = False
def getXBRL(nyear):
  noreport = False
  #Get all Companyfilings reported to the SEC 
  Feed = feedparser.parse(requests.get("https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK={}&type=10-k%25&dateb=&owner=exclude&start=0&count=40&output=atom".format(ncikkey)).text)
  #Get the Year the User specified before
  global IndexofYear
  IndexofYear = None
  for index in range(len(Feed.entries)):
      typeofreport = Feed.entries[index]['filing-type']
      reportedyear = datetime.strptime(Feed.entries[index]['filing-date'], '%Y-%d-%M').year - 1
      if reportedyear == nyear and typeofreport == '10-K':
        IndexofYear = index
        noreport = False
        break
      else:
        noreport = True
  if noreport == True or len(Feed.entries) == 0:
    if os.path.exists(ncikkey+'/'+str(nyear)):
      shutil.rmtree(ncikkey+'/'+str(nyear))
  else:
    time.sleep(1)
    #Build URL to access DIrectory AND Store Accession Number
    acn = Feed.entries[IndexofYear]['accession-number'].replace("-","")
    createdlink = 'https://www.sec.gov/Archives/edgar/data/{}/{}/'.format(ncikkey, acn)
    #print(createdlink)
    htmlroot = requests.get(createdlink).text
    htmlroot = LH.fromstring(htmlroot)
    link = htmlroot.xpath('.//a[contains(text(),".zip")]/@href')
    if len(link) == 0:
      noreport = True
    else:
      link = link[0]
      noreport = False
    if noreport == True:
      if os.path.exists(ncikkey+'/'+str(nyear)):
        shutil.rmtree(ncikkey+'/'+str(nyear))
    else:
      time.sleep(1)
      #Download XBRL Files
      if not os.path.exists(ncikkey):
          os.makedirs(ncikkey)
      if not os.path.exists(ncikkey+'/'+str(nyear)):
          os.makedirs(ncikkey+'/'+str(nyear))
      zipfiledl = requests.get('https://www.sec.gov/' + link)
      # Split URL to get the file name
      filename = link.split('/')[-1]
  
      # Writing the file to the local file system
      with open(ncikkey +'/' + str(nyear) + '/' + filename,'wb') as output_file:
        output_file.write(zipfiledl.content)
      #zip = zipfile.ZipFile(io.BytesIO(zipfiledl.content))
      #zip.extractall(ncikkey+'/'+str(nyear))
      XBRLFILE = filename

      #for element in XBRLFILE:
        #if  '_cal' not in element and '_def' not in element and '_lab' not in element and '_pre' not in element and '.xml' in element or '.htm' in element and '10k'in element or '.htm' in element and '10q_'in element or '.htm' in element and 'exhibit' not in element and str(nyear) in element: 
          #XBRLFILE = element

      os.system('python /content/Arelle/arelleCmdLine.py -f /content/'+ncikkey+'/'+str(nyear)+'/'+str(XBRLFILE)+' -v --packages /content/edgarTaxonomiesPackage.zip --factTable /content/'+ncikkey+'/'+str(nyear)+'/parsedXBRL.xml')

def parseXBRLtoJson(nyear):  
  tree = etree.parse(ncikkey+'/'+str(nyear)+'/parsedXBRL.xml')
  root = tree.getroot()
  try:
    year = root.xpath('.//facts[@concept="dei:DocumentFiscalYearFocus"]')
    year = year[0].getchildren()[0].text
  except:
    year = nyear
  
  try:
    form = root.xpath('.//facts[@concept="dei:DocumentType"]')
    form = form[0].getchildren()[0].text
  except:
    form = formtype
  
  try:
    cik = root.xpath('.//facts[@concept="dei:EntityCentralIndexKey"]')
    cik = cik[0].getchildren()[0].text
  except:
     cik = ncikkey

  try:
    ticker = root.xpath('.//facts')
    for x in ticker:
      if 'us-gaap:' not in x.attrib['concept'] and 'dei:' not in x.attrib['concept'] and 'srt:' not in x.attrib['concept'] and 'invest:' not in x.attrib['concept']:
        ticker = x.attrib['concept'][:4]
        if ':' in ticker:
          ticker = ticker.replace(':','')
        break;
  except:
      ticker = 'N/A'

  all_descendants = list(root.iter())
  elements = root.xpath('.//facts[@concept]|.//linkRole')
  jsonarr = []
  i = -1
  f = 0
  for x in elements:
    if x.tag == 'linkRole':
      f = 0
      i = i+1
      jsonarr.append({'TableName': x.getchildren()[0].attrib['concept'],'Form': form, 'Ticker': ticker,'CIK': cik, 'FY': year, 'Elements':[]})
    else:
      jsonarr[i]['Elements'].append({'Name':x.attrib['concept'], 'Values': {}})
      jsonarr[i]['Elements'][f]['Values'] = []
      for child in x:
        splitattrb = child.tag.split('_')
        cdtob = 'undefined'
        for x in splitattrb:
          if year in x:
            splitattrbval = x[x.find(year):x.find(year) + len(year)]
            break;
          else:
            splitattrbval = 'undefined'
        
        if splitattrbval == 'undefined':
          for x in splitattrb:  
            if str(int(year) - 1) in x: 
              splitattrbval = x[x.find(str(int(year) - 1)):x.find(str(int(year) - 1)) + len(str(int(year) - 1))]
              break;
        if splitattrbval == 'undefined':
          for x in splitattrb:  
            if str(int(year) - 2) in x:
              splitattrbval = x[x.find(str(int(year) - 2)):x.find(str(int(year) - 2)) + len(str(int(year) - 2))]
              break;
        if form == '10-K' and splitattrbval != 'undefined':
    
        
          cdtob = datetime.strptime(str(splitattrbval), '%Y') - timedelta(days=365)
          cdtob = cdtob.strftime('%Y')
        elif form == '10-Q' and splitattrbval != 'undefined': 
          print('Coming soon')
      
        jsonarr[i]['Elements'][f]['Values'].append({'id': child.tag, 'Value': child.text, 'fromFY': cdtob, 'toFY': splitattrbval})
      f = f+1
  li = []
  for elementtable in list(jsonarr):
    for element in list(elementtable['Elements']):
      if element['Values'] == [] or element['Values'][0]['id'] == 'facts':
        jsonarr[jsonarr.index(elementtable)]['Elements'].remove(element)
  jscp = jsonarr
  jsonString = json.dumps(jscp)
  jsonFile = open(ncikkey+'/'+str(nyear)+'/'+"XBRL.json", "w")
  jsonFile.write(jsonString)
  jsonFile.close()
  return jscp

for ncikkey in cikkey_list:
  yearsfetched = []
  if everyyear == True:
    current_year = date.today().year
    year_list = range(2009,current_year)
    for nyear in year_list:
      if not os.path.exists(ncikkey+'/'+str(nyear)):
        getXBRL(nyear)
        if os.path.exists(ncikkey+'/'+str(nyear)+'/parsedXBRL.xml'):
          parseXBRLtoJson(str(nyear))
          yearsfetched.append(str(nyear))
      else:
        if os.path.exists(ncikkey+'/'+str(nyear)+'/XBRL.json'):
          yearsfetched.append(str(nyear))
          continue
        elif not os.path.exists(ncikkey+'/'+str(nyear)+'/parsedXBRL.xml') and noreport == False:
          getXBRL(nyear)
          jscp = parseXBRLtoJson(str(nyear))
        elif not os.path.exists(ncikkey+'/'+str(nyear)+'/XBRL.json') and noreport == False:
          parseXBRLtoJson(str(nyear))
  elif year < 2009:
    #Check if Year is in the Range of Supported Years
    print('Year is not supported by this Script please enter a Year above 2008')
    sys.exit()
  else:
    getXBRL(year)
    parseXBRLtoJson(year)
    if os.path.exists(ncikkey+'/'+str(nyear)+'/XBRL.json') and noreport == False:
      yearsfetched = yearsfetched.append(str(nyear))
  substring = ','.join(yearsfetched)
  print('Got Doucments of: ' + ncikkey + ' for the following years: ' + substring)

Got Doucments of: 000320193 for the following years: 2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021


In [40]:

import pandas as pd
data = pd.DataFrame(pd.read_json('/content/000012927/2022/XBRL.json'))
data

,TableName,Form,Ticker,CIK,FY,Elements
0,dei:CoverAbstract,10-K,baA,12927,2022,"[{'Name': 'dei:DocumentType', 'Values': [{'id'..."
1,ba:AuditorInformationAbstract,10-K,baA,12927,2022,"[{'Name': 'dei:AuditorName', 'Values': [{'id':..."
2,us-gaap:IncomeStatementAbstract,10-K,baA,12927,2022,"[{'Name': 'us-gaap:Revenues', 'Values': [{'id'..."
3,us-gaap:StatementOfIncomeAndComprehensiveIncom...,10-K,baA,12927,2022,"[{'Name': 'us-gaap:ProfitLoss', 'Values': [{'i..."
4,us-gaap:StatementOfIncomeAndComprehensiveIncom...,10-K,baA,12927,2022,[{'Name': 'us-gaap:OtherComprehensiveIncomeUnr...
...,...,...,...,...,...,...
136,us-gaap:SegmentReportingAbstract,10-K,baA,12927,2022,[{'Name': 'us-gaap:RevenueFromContractWithCust...
137,us-gaap:SegmentReportingAbstract,10-K,baA,12927,2022,"[{'Name': 'us-gaap:ShareBasedCompensation', 'V..."
138,us-gaap:SegmentReportingAbstract,10-K,baA,12927,2022,"[{'Name': 'us-gaap:Assets', 'Values': [{'id': ..."
139,us-gaap:SegmentReportingAbstract,10-K,baA,12927,2022,[{'Name': 'us-gaap:PaymentsToAcquirePropertyPl...
